In [44]:
# 01. Import thư viện
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.models import *
from sklearn.ensemble import RandomForestClassifier

# 02. Load dữ liệu
path = "../data/processed/BankChurners_processed.csv"
data = np.loadtxt(path, delimiter=",", skiprows=1)
X = data[:, :-1]
y = data[:, -1].astype(int)
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print("Label phân bố:", np.unique(y, return_counts=True))


Shape X: (10127, 36)
Shape y: (10127,)
Label phân bố: (array([0, 1]), array([8500, 1627], dtype=int64))


In [45]:
# 03. Chia Train - Test (NumPy)
X_train, X_test, y_train, y_test = train_test_split_numpy(
    X, y, test_size=0.2
)
print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])
print("Phân bố label train:", np.unique(y_train, return_counts=True))
print("Phân bố label test:", np.unique(y_test, return_counts=True))


Train size: 8102
Test size: 2025
Phân bố label train: (array([0, 1]), array([6802, 1300], dtype=int64))
Phân bố label test: (array([0, 1]), array([1698,  327], dtype=int64))


In [46]:
# 04. Chuẩn hóa dữ liệu
scaler = NumpyScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Mean feature 0 (train):", np.mean(X_train_scaled[:, 0]))
print("Std feature 0 (train):", np.std(X_train_scaled[:, 0]))


Mean feature 0 (train): -4.209584215809036e-16
Std feature 0 (train): 1.0000000000000362


In [47]:
# 05. Logistic Regression tự cài bằng NumPy
my_model = NumpyLogisticRegression(
    learning_rate=0.1,
    n_iterations=2000
)
my_model.fit(X_train_scaled, y_train)
y_pred_custom = my_model.predict(X_test_scaled)
print("- Kết quả Logistic Regression (NumPy)")
evaluate_model(y_test, y_pred_custom)


- Kết quả Logistic Regression (NumPy)
Accuracy: 0.9066666666666666
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      1698
           1       0.79      0.58      0.67       327

    accuracy                           0.91      2025
   macro avg       0.86      0.77      0.81      2025
weighted avg       0.90      0.91      0.90      2025

Confusion Matrix:
 [[1647   51]
 [ 138  189]]


In [48]:
# 06. Random Forest (Sklearn Baseline)
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("- Kết quả Random Forest")
evaluate_model(y_test, y_pred_rf)


- Kết quả Random Forest
Accuracy: 0.9624691358024692
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1698
           1       0.93      0.83      0.88       327

    accuracy                           0.96      2025
   macro avg       0.95      0.91      0.93      2025
weighted avg       0.96      0.96      0.96      2025

Confusion Matrix:
 [[1678   20]
 [  56  271]]


## Tổng kết và Đánh giá Mô hình 
---

Tổng kết lại quá trình xây dựng mô hình dự đoán khách hàng rời bỏ, chỉ sử dụng thư viện NumPy và so sánh hiệu quả với mô hình chuẩn Random Forest của thư viện Scikit-learn.

#### 1. So sánh Hiệu năng (Performance)

Chúng ta đánh giá hai mô hình dựa trên các tiêu chí chính:

`Logistic Regression:`

- Độ chính xác (Accuracy): Đạt khoảng ~90%.

- Đặc điểm: Đây là mô hình tuyến tính. Kết quả này cho thấy dữ liệu có tính phân tách khá tốt sau khi đã chuẩn hóa.

- Ưu điểm: Tốc độ huấn luyện cực nhanh nhờ tính toán ma trận, code nhẹ, dễ kiểm soát tham số.

- Nhược điểm: Chỉ số Recall cho nhóm rời bỏ (Class 1) thấp hơn so với Random Forest, do hạn chế trong việc học các mối quan hệ phức tạp/phi tuyến tính.

`Random Forest:`

- Độ chính xác (Accuracy): Đạt mức cao hơn, khoảng 96%.

- Đặc điểm: Mô hình phi tuyến tính (Non-linear).

- Ưu điểm: Chỉ số Recall cho lớp khách hàng rời bỏ cao hơn hẳn. Điều này rất quan trọng trong bài toán thực tế vì ngân hàng sợ "bỏ sót" khách hàng sắp đi hơn là "nhầm lẫn".

- Lý do: Random Forest có khả năng tự động bắt được các tương tác phức tạp giữa các đặc trưng (ví dụ: sự kết hợp giữa số lần giao dịch giảm và hạn mức tín dụng cao).

#### 2. Phát hiện và khuyến nghị giải pháp

Dựa trên trọng số ($w$) của Logistic Regression và mức độ quan trọng của biến (Feature Importance), ta rút ra các yếu tố ảnh hưởng đến khách hàng sắp rời bỏ:

1.  Total_Trans_Ct (Tổng số lần giao dịch): Yếu tố quan trọng nhất. Khách hàng sắp rời bỏ có dấu hiệu "ngủ đông" - tần suất quẹt thẻ giảm đột ngột.

2.  Total_Revolving_Bal (Số dư nợ quay vòng): Khách hàng có ý định đóng thẻ thường sẽ thanh toán hết nợ tồn đọng (số dư về 0) trước khi ra quyết định.

3.  Total_Trans_Amt (Tổng tiền giao dịch): Tương quan thuận với số lần giao dịch, chi tiêu giảm mạnh là tín hiệu báo động đỏ.

-> Giải pháp khuyến nghị: Ngân hàng cần thiết lập hệ thống cảnh báo sớm. Nếu một khách hàng có `Total_Trans_Ct` tụt xuống dưới mức trung bình (ví dụ: < 40 lần/năm) và `Revolving_Bal` thấp, hệ thống cần tự động gửi ưu đãi (hoàn tiền, giảm lãi suất) để giữ chân khách hàng này ngay lập tức.
